In [1]:
###################################################################################
##
## From https://github.com/keras-team/keras/blob/master/examples/cifar10_resnet.py
##
###################################################################################

from __future__ import print_function
import keras
import keras.layers as layers
from keras.layers import Dense, Conv2D, BatchNormalization, Activation, Dropout
from keras.layers import GlobalAveragePooling2D, MaxPooling2D, AveragePooling2D, Input, Flatten
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10304915316030050895
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3428974592
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14487929971298353111
physical_device_desc: "device: 0, name: GeForce GTX 970, pci bus id: 0000:01:00.0, compute capability: 5.2"
]


In [3]:
# Training parameters
batch_size = 128  # orig paper trained all networks with batch_size=128
epochs = 100
data_augmentation = True
num_classes = 10

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

# Depth of the model
depth = 121

# Growth rate of the dense blocks
growth_rate = 32

# Model name
model_type = 'DenseNet-%d' % (depth)

In [4]:
# Load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [5]:
# Input image dimensions.
input_shape = x_train.shape[1:]

# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# If subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 1)


In [6]:
depths = {
    121: [6, 12, 24, 16]
}

def lr_schedule(epoch):

    lr = 1e-1
    if epoch > 80:
        lr *= 0.5e-3
    elif epoch > 60:
        lr *= 1e-3
    elif epoch > 40:
        lr *= 1e-2
    elif epoch > 20:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

def dense_block(x, blocks, name):
    """A dense block.
    # Arguments
        x: input tensor.
        blocks: integer, the number of building blocks.
        name: string, block label.
    # Returns
        output tensor for the block.
    """
    for i in range(blocks):
        x = conv_block(x, 32, name=name + '_block' + str(i + 1))
    return x


def transition_block(x, reduction, name):
    """A transition block.
    # Arguments
        x: input tensor.
        reduction: float, compression rate at transition layers.
        name: string, block label.
    # Returns
        output tensor for the block.
    """
    x = layers.BatchNormalization(
        epsilon=1.001e-5,
        name=name + '_bn'
    )(x)
    x = layers.Activation('relu', name=name + '_relu')(x)
    x = layers.Conv2D(
        int(x.shape[3] * reduction), 
        1,
        use_bias=False,
        name=name + '_conv'
    )(x)
    x = layers.AveragePooling2D(2, strides=2, name=name + '_pool')(x)
    return x


def conv_block(x, growth_rate, name):
    x1 = layers.BatchNormalization(
        epsilon=1.001e-5,
        name=name + '_0_bn'
    )(x)
    x1 = layers.Activation('relu', name=name + '_0_relu')(x1)
    x1 = layers.Conv2D(
        4 * growth_rate, 1,
        use_bias=False,
        name=name + '_1_conv'
    )(x1)
    
    x1 = layers.BatchNormalization(
        epsilon=1.001e-5,
        name=name + '_1_bn'
    )(x1)
    x1 = layers.Activation('relu', name=name + '_1_relu')(x1)
    x1 = layers.Conv2D(
        growth_rate, 3,
        padding='same',
        use_bias=False,
        name=name + '_2_conv'
    )(x1)
    x = layers.Concatenate(name=name + '_concat')([x, x1])
    return x

def densenet(input_shape, blocks, num_classes=10):
    inputs = layers.Input(shape=input_shape)
    
    x = layers.ZeroPadding2D(padding=((3, 3), (3, 3)))(inputs)
    x = layers.Conv2D(64, 7, strides=2, use_bias=False, name='conv1/conv')(x)
    x = layers.BatchNormalization(epsilon=1.001e-5, name='conv1/bn')(x)
    x = layers.Activation('relu', name='conv1/relu')(x)
    x = layers.ZeroPadding2D(padding=((1, 1), (1, 1)))(x)
    x = layers.MaxPooling2D(3, strides=2, name='pool1')(x)

    x = dense_block(x, blocks[0], name='conv2')
    x = transition_block(x, 0.5, name='pool2')
    x = dense_block(x, blocks[1], name='conv3')
    x = transition_block(x, 0.5, name='pool3')
    x = dense_block(x, blocks[2], name='conv4')
    x = transition_block(x, 0.5, name='pool4')
    x = dense_block(x, blocks[3], name='conv5')

    x = layers.BatchNormalization(epsilon=1.001e-5, name='bn')(x)
    x = layers.Activation('relu', name='relu')(x)

    x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
    x = layers.Dense(num_classes, activation='softmax', name='fc1000')(x)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=x)
    
    return model

In [7]:
model = densenet(input_shape, depths[121])

model.compile(loss='categorical_crossentropy',
              optimizer=SGD(learning_rate=lr_schedule(0)),
              metrics=['accuracy'])

model.summary()
print(model_type)

Learning rate:  0.1
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 16, 16, 64)   9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1/conv[0][0]                 
__________________________________________________________________________

In [8]:
# Prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

In [9]:
# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

In [11]:
# Run training, with or without data augmentation.
if not data_augmentation:
    print('Not using data augmentation.')
    history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=20,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.1,
        # set range for random zoom
        zoom_range=0.1,
        # set range for random channel shifts
        channel_shift_range=0.1,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # randomly flip images
        horizontal_flip=True)

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                        validation_data=(x_test, y_test),
                        epochs=epochs, verbose=1, workers=7, use_multiprocessing=True,                        
                        callbacks=callbacks)

Using real-time data augmentation.
Epoch 1/100
Learning rate:  0.1
391/391 [==============================] - 53s 108ms/step - loss: 2.1645 - accuracy: 0.2934 - val_loss: 1.5190 - val_accuracy: 0.4362

Epoch 00001: val_accuracy improved from -inf to 0.43620, saving model to /home/vicdoja/repos/ComputerVision-Labs/densenet/saved_models/cifar10_DenseNet-121_model.001.h5
Epoch 2/100
Learning rate:  0.1
391/391 [==============================] - 38s 95ms/step - loss: 1.5855 - accuracy: 0.4485 - val_loss: 1.3891 - val_accuracy: 0.5357

Epoch 00002: val_accuracy improved from 0.43620 to 0.53570, saving model to /home/vicdoja/repos/ComputerVision-Labs/densenet/saved_models/cifar10_DenseNet-121_model.002.h5
Epoch 3/100
Learning rate:  0.1
391/391 [==============================] - 38s 95ms/step - loss: 1.3514 - accuracy: 0.5176 - val_loss: 1.2982 - val_accuracy: 0.5545

Epoch 00003: val_accuracy improved from 0.53570 to 0.55450, saving model to /home/vicdoja/repos/ComputerVision-Labs/densenet/

Epoch 12/100
Learning rate:  0.1
391/391 [==============================] - 37s 95ms/step - loss: 0.7441 - accuracy: 0.7375 - val_loss: 0.8214 - val_accuracy: 0.7268

Epoch 00012: val_accuracy did not improve from 0.74310
Epoch 13/100
Learning rate:  0.1
391/391 [==============================] - 38s 95ms/step - loss: 0.7109 - accuracy: 0.7487 - val_loss: 0.6889 - val_accuracy: 0.7608

Epoch 00013: val_accuracy improved from 0.74310 to 0.76080, saving model to /home/vicdoja/repos/ComputerVision-Labs/densenet/saved_models/cifar10_DenseNet-121_model.013.h5
Epoch 14/100
Learning rate:  0.1
391/391 [==============================] - 38s 95ms/step - loss: 0.6708 - accuracy: 0.7633 - val_loss: 0.6976 - val_accuracy: 0.7569

Epoch 00014: val_accuracy did not improve from 0.76080
Epoch 15/100
Learning rate:  0.1
391/391 [==============================] - 38s 96ms/step - loss: 0.6540 - accuracy: 0.7699 - val_loss: 0.7367 - val_accuracy: 0.7496

Epoch 00015: val_accuracy did not improve from 0.7

391/391 [==============================] - 38s 95ms/step - loss: 0.4241 - accuracy: 0.8515 - val_loss: 0.5783 - val_accuracy: 0.8066

Epoch 00023: val_accuracy did not improve from 0.80660
Epoch 24/100
Learning rate:  0.010000000000000002
391/391 [==============================] - 38s 95ms/step - loss: 0.4158 - accuracy: 0.8542 - val_loss: 0.5745 - val_accuracy: 0.8094

Epoch 00024: val_accuracy improved from 0.80660 to 0.80940, saving model to /home/vicdoja/repos/ComputerVision-Labs/densenet/saved_models/cifar10_DenseNet-121_model.024.h5
Epoch 25/100
Learning rate:  0.010000000000000002
391/391 [==============================] - 38s 95ms/step - loss: 0.4028 - accuracy: 0.8572 - val_loss: 0.5695 - val_accuracy: 0.8110

Epoch 00025: val_accuracy improved from 0.80940 to 0.81100, saving model to /home/vicdoja/repos/ComputerVision-Labs/densenet/saved_models/cifar10_DenseNet-121_model.025.h5
Epoch 26/100
Learning rate:  0.010000000000000002
391/391 [==============================] - 38s 95

391/391 [==============================] - 37s 94ms/step - loss: 0.3456 - accuracy: 0.8782 - val_loss: 0.5878 - val_accuracy: 0.8113

Epoch 00035: val_accuracy did not improve from 0.81600
Epoch 36/100
Learning rate:  0.010000000000000002
391/391 [==============================] - 37s 95ms/step - loss: 0.3474 - accuracy: 0.8789 - val_loss: 0.5866 - val_accuracy: 0.8132

Epoch 00036: val_accuracy did not improve from 0.81600
Epoch 37/100
Learning rate:  0.010000000000000002
391/391 [==============================] - 37s 95ms/step - loss: 0.3414 - accuracy: 0.8789 - val_loss: 0.5856 - val_accuracy: 0.8138

Epoch 00037: val_accuracy did not improve from 0.81600
Epoch 38/100
Learning rate:  0.010000000000000002
391/391 [==============================] - 37s 94ms/step - loss: 0.3332 - accuracy: 0.8820 - val_loss: 0.5944 - val_accuracy: 0.8111

Epoch 00038: val_accuracy did not improve from 0.81600
Epoch 39/100
Learning rate:  0.010000000000000002
391/391 [==============================] - 3

391/391 [==============================] - 37s 95ms/step - loss: 0.3072 - accuracy: 0.8910 - val_loss: 0.5877 - val_accuracy: 0.8151

Epoch 00048: val_accuracy did not improve from 0.81600
Epoch 49/100
Learning rate:  0.001
391/391 [==============================] - 37s 94ms/step - loss: 0.3111 - accuracy: 0.8908 - val_loss: 0.5890 - val_accuracy: 0.8139

Epoch 00049: val_accuracy did not improve from 0.81600
Epoch 50/100
Learning rate:  0.001
391/391 [==============================] - 37s 94ms/step - loss: 0.3137 - accuracy: 0.8884 - val_loss: 0.5899 - val_accuracy: 0.8140

Epoch 00050: val_accuracy did not improve from 0.81600
Epoch 51/100
Learning rate:  0.001
391/391 [==============================] - 37s 94ms/step - loss: 0.3073 - accuracy: 0.8900 - val_loss: 0.5921 - val_accuracy: 0.8142

Epoch 00051: val_accuracy did not improve from 0.81600
Epoch 52/100
Learning rate:  0.001
391/391 [==============================] - 37s 94ms/step - loss: 0.3086 - accuracy: 0.8913 - val_loss: 0

391/391 [==============================] - 36s 92ms/step - loss: 0.3025 - accuracy: 0.8923 - val_loss: 0.5927 - val_accuracy: 0.8156

Epoch 00061: val_accuracy did not improve from 0.81600
Epoch 62/100
Learning rate:  0.0001
391/391 [==============================] - 36s 92ms/step - loss: 0.2992 - accuracy: 0.8943 - val_loss: 0.5906 - val_accuracy: 0.8151

Epoch 00062: val_accuracy did not improve from 0.81600
Epoch 63/100
Learning rate:  0.0001
391/391 [==============================] - 37s 92ms/step - loss: 0.3022 - accuracy: 0.8935 - val_loss: 0.5913 - val_accuracy: 0.8148

Epoch 00063: val_accuracy did not improve from 0.81600
Epoch 64/100
Learning rate:  0.0001
391/391 [==============================] - 37s 93ms/step - loss: 0.3097 - accuracy: 0.8901 - val_loss: 0.5890 - val_accuracy: 0.8157

Epoch 00064: val_accuracy did not improve from 0.81600
Epoch 65/100
Learning rate:  0.0001
391/391 [==============================] - 37s 93ms/step - loss: 0.3076 - accuracy: 0.8926 - val_los

391/391 [==============================] - 37s 93ms/step - loss: 0.3050 - accuracy: 0.8928 - val_loss: 0.5888 - val_accuracy: 0.8155

Epoch 00074: val_accuracy did not improve from 0.81600
Epoch 75/100
Learning rate:  0.0001
391/391 [==============================] - 37s 93ms/step - loss: 0.2962 - accuracy: 0.8939 - val_loss: 0.5914 - val_accuracy: 0.8148

Epoch 00075: val_accuracy did not improve from 0.81600
Epoch 76/100
Learning rate:  0.0001
391/391 [==============================] - 37s 93ms/step - loss: 0.3060 - accuracy: 0.8950 - val_loss: 0.5926 - val_accuracy: 0.8139

Epoch 00076: val_accuracy did not improve from 0.81600
Epoch 77/100
Learning rate:  0.0001
391/391 [==============================] - 37s 93ms/step - loss: 0.3005 - accuracy: 0.8931 - val_loss: 0.5903 - val_accuracy: 0.8147

Epoch 00077: val_accuracy did not improve from 0.81600
Epoch 78/100
Learning rate:  0.0001
391/391 [==============================] - 37s 93ms/step - loss: 0.3084 - accuracy: 0.8909 - val_los

Process Keras_worker_ForkPoolWorker-1182:
Process Keras_worker_ForkPoolWorker-1180:
Process Keras_worker_ForkPoolWorker-1183:
Process Keras_worker_ForkPoolWorker-1178:
Process Keras_worker_ForkPoolWorker-1177:
Process Keras_worker_ForkPoolWorker-1181:
  File "/home/vicdoja/anaconda3/envs/tf_env/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Process Keras_worker_ForkPoolWorker-1179:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/vicdoja/anaconda3/envs/tf_env/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/vicdoja/anaconda3/envs/tf_env/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/vicdoja/anaconda3/envs/tf_env/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  F

KeyboardInterrupt: 

  File "/home/vicdoja/anaconda3/envs/tf_env/lib/python3.8/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt


In [ ]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
from matplotlib import pyplot as plt

#  "Accuracy"
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# "Loss"
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()